# Part 1: Data Cleaning

#### Author: Ali Eddeb

In this notebook, I will be loading and cleaning an Amazon review dataset for use in this project. To reiterate, the goal of this project is to spot fake reviews.

In [1]:
#importing pandas
import pandas as pd

### 1. Data Loading
I will start by loading in the data. As the whole dataset is incredibly large (34 gb!), I will only work with a smaller subset. Specifically, I will work with reviews for just the Beauty product category.

In [2]:
#let's work with one set of reviews for the beauty category; it is in a compressed .gz format so will need to unzip using gzip
beauty_df = pd.read_json('All_Beauty.json.gz', compression='gzip', lines=True)

In [3]:
#how big is the dataset?
beauty_df.shape

(371345, 12)

We can see the Amazon review dataset for the beauty product category has 371,345 reviews and 12 features.

In [5]:
#let's see what was loaded
beauty_df.tail(3)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
371342,5,True,"03 1, 2017",A1B5DK6CTP2P24,B01HJEGTYK,Norma Jennings,Makes me look good fast.,Five Stars,1488326400,46,NaN,NaN
371343,2,True,"02 21, 2017",A23OUYS5IRMJS9,B01HJEGTYK,Lee,Way lighter than photo\nNot mix blend of color...,Ok but color way off and volume as well,1487635200,NaN,NaN,NaN
371344,2,True,"12 15, 2016",A24KQ9RVU81L87,B01HJEGTYK,Season341,No return instructions/phone # in packaging. ...,Might return for a replacement if I could.,1481760000,NaN,NaN,NaN


Each row represents a review for a product. Here is a quick rundown of what each feature represents based on my research:
- overall - represents the customer's rating of the product, on a scale from 1 to 5
- verified - represents whether or not the review is associated with a verified purchase of the product
- reviewTime - the date the review was published on Amazon 
- reviewerID - a unique ID for the customer that is submitting the review
- asin - a unique ID for the product that is being reviewed
- reviewerName - the customer's name 
- reviewText - the customer's review
- summary - the title for the review which is also submitted by the customer 
- unixReviewTime - the date the review was published on Amazon in unix time
- vote - number of votes from other people that found the review helpful
- style - the version or style of the product (this is a product specific field) 
- image - url link(s) to image(s) of the product (this is a product specific field) 

### 2. Data Cleaning
Let's take a look at the data and see what needs to be cleaned.

#### 2.1 Null Values

In [7]:
#check percentage of nulls per column
beauty_df.isna().sum()/len(beauty_df)

overall           0.000000
verified          0.000000
reviewTime        0.000000
reviewerID        0.000000
asin              0.000000
reviewerName      0.000102
reviewText        0.001074
summary           0.000555
unixReviewTime    0.000000
vote              0.860240
style             0.660806
image             0.977404
dtype: float64

Features with null values are: reviewerName, reviewText, summary, vote, style, and image. Here is what I will do for each:

- reviewerName - I noticed that there are some customers that are named 'Amazon Customer'. I am assuming these individuals did not fill in their name and the default value was 'Amazon Customer'. Therefore, for any nulls, **I will fill them in with 'Amazon Customer'** to indicate they are unnamed.
- reviewText - The whole purpose of this project is to explore the text in the reviews. For that reason, **any rows with no review text will be removed**. Luckily, there are very few rows that are missing review text.
- summary - Because the summary is a reflection of the reviewText, for any rows that are missing a summary, **I will fill them in with the text in the reviewText field**. Note: there is a very low percentage of NaNs in the summary field (<0.06%) so this should not greatly affect 
- vote - There are a lot of missing values in this column (86%). After visiting Amazon's website and looking at some reviews, I noticed if a review was found helpful (ie. it had votes), there was an html tag for this variable. If no one found the review helpful, there was no html tag. What this means is if there is null value, it actually means there were 0 votes for the review. Therefore, **I will fill missing values with 0**. 
- style - there is a high percentage of missing values in this field and it is also a product specific field (ie. irrelevant). For those reasons, **I will remove this feature entirely**.
- image - there is a high percentage of missing values in this field and it is also a product specific field (ie. irrelevant). For those reasons, **I will remove this feature entirely**.

#### 2.2 Data Types

In [6]:
#Let's check the data types of each feature
beauty_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371345 entries, 0 to 371344
Data columns (total 12 columns):
overall           371345 non-null int64
verified          371345 non-null bool
reviewTime        371345 non-null object
reviewerID        371345 non-null object
asin              371345 non-null object
reviewerName      371307 non-null object
reviewText        370946 non-null object
summary           371139 non-null object
unixReviewTime    371345 non-null int64
vote              51899 non-null object
style             125958 non-null object
image             8391 non-null object
dtypes: bool(1), int64(2), object(9)
memory usage: 31.5+ MB


After reviewing the columns, I have found that the following columns do not have a suitable dtype and need to be adjusted:
- verified - is currently boolean. In order to be used in the analysis, it needs to be numeric. I will convert this feature to **binary**.
- reviewTime - needs to be converted to type **datetime**
- vote - needs to be converted to type **int**

Note: I will not be converting unixReviewTime because I will be removing this feature entirely so no need for wasted efforts. I am removing the unixReviewTime feature entirely because it is redundant as we already have the reviewTime column which contains the same information. I did a check (not shown here) and both reviewTime and unixReviewTime only contain dates but no time data.

#### Function for cleaning

Now I will aggregate all that was mentioned above and create a function to clean the dataframe.

In [7]:
def cleanDF(dataframe):
    '''
    Pass in a dataframe that contains an amazon review dataset to be cleaned.
    Assumption: dataframe was loaded in and no manipulation has occured.
    
    Returns cleaned dataframe
    '''
    #drop 'style', 'image' and 'unixReviewTime' columns
    modified_df = dataframe.drop(['style','image', 'unixReviewTime'], axis = 1)
    
    #remove rows with no review text
    modified_df = modified_df.dropna(axis = 0, subset = ['reviewText'])
    
    #first need to remove commas from vote column in order to convert to type int
    modified_df['vote'] = modified_df['vote'].str.replace(',','')
    #fill NaNs in vote column with zeroes and change votes from type object to int
    modified_df['vote'] = modified_df['vote'].fillna(0).astype(int)
    
    #fill empty summaries with review text
    modified_df['summary'].fillna(modified_df['reviewText'], inplace = True)
    
    #fill empty names with 'Amazon Customer'
    modified_df['reviewerName'].fillna('Amazon Customer', inplace = True)
    
    #change review time to type datetime for later modifications
    modified_df = modified_df.astype({'reviewTime': 'datetime64[ns]'})
    
    #Convert True and False under verified purchase column to binary
    map_dict = {False:0, True:1}
    modified_df['verified'] = modified_df['verified'].map(map_dict)
    
    #need to reset index
    modified_df = modified_df.reset_index().drop('index', axis=1)
    
    return modified_df

Now let's use the function on the beauty dataframe in order to clean it.

In [8]:
#cleaning beauty_df and saving it to new dataframe called clean_beauty_df
clean_beauty_df = cleanDF(beauty_df)

In [9]:
#let's see what the cleaned df looks like
clean_beauty_df.tail(3)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,vote
370943,5,1,2017-03-01,A1B5DK6CTP2P24,B01HJEGTYK,Norma Jennings,Makes me look good fast.,Five Stars,46
370944,2,1,2017-02-21,A23OUYS5IRMJS9,B01HJEGTYK,Lee,Way lighter than photo\nNot mix blend of color...,Ok but color way off and volume as well,0
370945,2,1,2016-12-15,A24KQ9RVU81L87,B01HJEGTYK,Season341,No return instructions/phone # in packaging. ...,Might return for a replacement if I could.,0


Looks good! I will save the cleanDF function to python script called **functions_library** so that I can use this function for later notebooks.

### 3. Verification - is the data clean?

Lastly, before moving onto data exploration, I would like to verify that there are no missing values nor duplicates.

In [10]:
#let's check if there are any missing values
clean_beauty_df.isna().sum()

overall         0
verified        0
reviewTime      0
reviewerID      0
asin            0
reviewerName    0
reviewText      0
summary         0
vote            0
dtype: int64

Great! No more missing values.

#### Duplicates

In [11]:
#let's check if there are any duplicate rows
clean_beauty_df.duplicated().sum()

8694

We have 8694 duplicates! Let's take a look at some of the rows to see why this might be happening.

In [12]:
#let's look at a couple of duplicate rows
clean_beauty_df[clean_beauty_df.duplicated()].head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,vote
7161,5,1,2014-11-19,A3AMP8ZS2WQ94N,B0000530HU,Antonio Dias,ok,Five Stars,0
12325,5,1,2016-03-09,A3MW93I43BV8DD,B00011QUDE,michelle,excellent,Five Stars,0
13910,5,1,2017-12-29,A1GCZCBN67Q4AY,B0001AD4TS,Fred,my wife loves it,Five Stars,0
18388,5,1,2016-08-30,A3RPGLL3257S2X,B0002JHI1I,Nancy B,very satisfied,Five Stars,0
20379,5,1,2015-11-21,A2GVXFM3PB7EHV,B00070D21W,dennis paulk,very good,Five Stars,0


<br>
I'm going to look at some reviewer - product combinations to see why there are duplicates. Let's take a look at the first two reviewers (reviewerID: A3AMP8ZS2WQ94N, A3MW93I43BV8DD) and their associated products they reviewed (asin: B0000530HU, B00011QUDE).

In [13]:
#reviewer: A3AMP8ZS2WQ94N; asin: B0000530HU
clean_beauty_df[(clean_beauty_df['reviewerID'] == 'A3AMP8ZS2WQ94N') & (clean_beauty_df['asin'] == 'B0000530HU')]

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,vote
7160,5,1,2014-11-19,A3AMP8ZS2WQ94N,B0000530HU,Antonio Dias,ok,Five Stars,0
7161,5,1,2014-11-19,A3AMP8ZS2WQ94N,B0000530HU,Antonio Dias,ok,Five Stars,0


In [14]:
#reviewer: A3MW93I43BV8DD; asin: B00011QUDE
clean_beauty_df[(clean_beauty_df['reviewerID'] == 'A3MW93I43BV8DD') & (clean_beauty_df['asin'] == 'B00011QUDE')]

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,vote
12324,5,1,2016-03-09,A3MW93I43BV8DD,B00011QUDE,michelle,excellent,Five Stars,0
12325,5,1,2016-03-09,A3MW93I43BV8DD,B00011QUDE,michelle,excellent,Five Stars,0


In both cases, they have the same review listed twice in the dataset, but there is no clear reason for duplication. 

Let's go back to the original dataset (pre-cleaning) to see what might be the cause.

In [15]:
#reviewer: A3AMP8ZS2WQ94N; asin: B0000530HU in original dataset
beauty_df[(beauty_df['reviewerID'] == 'A3AMP8ZS2WQ94N') & (beauty_df['asin'] == 'B0000530HU')]

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
7165,5,True,"11 19, 2014",A3AMP8ZS2WQ94N,B0000530HU,Antonio Dias,ok,Five Stars,1416355200,NaN,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",NaN
7166,5,True,"11 19, 2014",A3AMP8ZS2WQ94N,B0000530HU,Antonio Dias,ok,Five Stars,1416355200,NaN,NaN,NaN


In [16]:
#reviewer: A3MW93I43BV8DD; asin: B00011QUDE in original dataset
beauty_df[(beauty_df['reviewerID'] == 'A3MW93I43BV8DD') & (beauty_df['asin'] == 'B00011QUDE')]

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
12330,5,True,"03 9, 2016",A3MW93I43BV8DD,B00011QUDE,michelle,excellent,Five Stars,1457481600,NaN,{'Style Name:': ' 1505'},NaN
12331,5,True,"03 9, 2016",A3MW93I43BV8DD,B00011QUDE,michelle,excellent,Five Stars,1457481600,NaN,{'Style Name:': ' 1507'},NaN


In both cases, we see that all the features are the same except for the style column. There were changes to the style column but we omitted the style column in our cleaning steps as it was deemed an irrelevant feature. So if we ignore the style column and note that all other features are the same for each review, we can, therefore, **remove the duplicates**. 

In [17]:
#removing duplicates
clean_beauty_df.drop_duplicates(inplace=True)

In [18]:
#verify there are no more duplicates and check how many rows & features exist in the cleaned dataset
print(f'Number of duplicates: {clean_beauty_df.duplicated().sum()}')
print(f'shape: {clean_beauty_df.shape}')

Number of duplicates: 0
shape: (362252, 9)


Great! No more duplicates and we now have 362,252 reviews (rows) and 9 features.

*I will update the cleanDF function in the functions_library to include the removal of duplicates.*

Please proceed to next notebook where I will perform data exploration.